In [16]:
import os
import pandas as pd
from pandas import read_csv
from pandas import DataFrame
from math import *
import math
from sympy import *
from sympy.abc import x,y,z,a,b,c,d
import numpy as np
import numpy
import csv
from math import e

def preprocessing(before_filename, after_filename):
    train_dataset = read_csv(before_filename, header=0, index_col=None)
    train_values = train_dataset.values

    power = train_values[:, 0]
    temperature = train_values[:, 1]
    humidity = train_values[:, 2]
    speed = train_values[:, 3]
    common = [1 for i in range(power.shape[0])]

    transfer_power = []
    transfer_temperature = []
    transfer_humidity = []
    transfer_speed = []

    except_value = -5.0

    for index in range(len(power)):
        if round(log(power[index], e), 2) == -inf:
            transfer_power.append(except_value)
        else:
            transfer_power.append(round(log(power[index], e), 2))

    for index in range(len(temperature)):
        if round(log(temperature[index], e), 2) == -inf:
            transfer_temperature.append(except_value)
        else:
            transfer_temperature.append(round(log(temperature[index], e), 2))

    for index in range(len(humidity)):
        if round(log(humidity[index], e), 2) == -inf:
            transfer_humidity.append(except_value)
        else:
            transfer_humidity.append(round(log(humidity[index], e), 2))

    for index in range(len(speed)):
        if round(log(speed[index], e), 2) == -inf:
            transfer_speed.append(except_value)
        else:
            transfer_speed.append(round(log(speed[index], e), 2))

    transfer_common = numpy.array(common)

    Data = {'power': transfer_power, 'temperature': transfer_temperature, 'humidity': transfer_humidity,
            'speed': transfer_speed, 'common': transfer_common}
    df = DataFrame(Data, columns=['power', 'temperature', 'humidity', 'speed', 'common'])
    df.to_csv(after_filename, index=False)

def train_start(train_filename):
    dt = read_csv(train_filename, header=0, index_col=False)
    df = pd.DataFrame(dt)
    i = 0
    while i < (len(df.values)/2):
    
        try:
            temp1 = df.sample(4, frac=None, replace=True, weights=None, random_state=None, axis=None)
            a = np.array(temp1.values[:, 1:])
            b = np.array(temp1.values[0:, 0])
            x = np.linalg.solve(a, b)
            dataframe = pd.DataFrame({'X1': x[0], 'X2': x[1], 'X3': x[2], 'X4': x[3]}, index=[i])
            dataframe.to_csv("Xee_A.csv", mode='a', header=None, index=True, sep=',')#Xee_A.csv...Xee_F.csv
            i = i + 1
            print('第%d组系数' % (i))
            print(x)
            total_mape = 0
            m = 2.7182818
            for line in range(len(df.values)):
                predict_log_values = np.dot(df.values[line][1:], x.reshape(4, 1))

                predict_value = pow(e, predict_log_values)

                mape = abs(pow(e, df.values[line][0]) - predict_value) / pow(e, df.values[line][0])
                total_mape = total_mape + mape

            average_mape = total_mape / len(df.values)
            print(average_mape)
            dataframe = pd.DataFrame({'X5': average_mape}, index=[i - 1])
            dataframe.to_csv("Xe1_A.csv", mode='a', header=None, index=True, sep=',')#Xe1_A.csv...Xe1_F.csv
        except:
            i = i - 1



def test_start(test_filename, n):
    dt = read_csv(test_filename, header=0, index_col=False)
    df = pd.DataFrame(dt)
    para = para_median(n)
    mape = 0
#    print(para)
    for index in range(len(df.values)):
        X = df.values[index][1:]
        testY = df.values[index][0]
        para = np.array(para).reshape(4, 1)
        predict = np.dot(X, para)
        mape = mape + abs(pow(e,testY)-pow(e,predict))/pow(e,testY)
    print(mape/len(df.values))

def para_median(n):
    lc = pd.DataFrame(pd.read_csv('Xe1_A.csv', header=None, names=['xulie', 'wucha']))#Xe1_A.csv...Xe1_F.csv
    tdd = lc.sort_values(by='wucha', ascending=True).head(n)
    index = list(tdd['xulie'])
    dt = pd.read_csv('Xee_A.csv', header=None, names=['xulie', '参数1', '参数2', '参数3', '参数4'])#Xee_A.csv...Xee_F.csv
    df = pd.DataFrame(dt)
    X = []
    for i in index:
        X.append([df.values[i][1],df.values[i][2],df.values[i][3],df.values[i][4]])
    X = np.array(X)
    para1 = np.median(X[:, 0])
    para2 = np.median(X[:, 1])
    para3 = np.median(X[:, 2])
    para4 = np.median(X[:, 3])
    return [para1, para2, para3, para4]


if __name__ == "__main__":
    preprocessing("trainingA.csv", "trainA.csv")#trainingA.csv/trainA.csv...trainingF.csv/trainF.csv
    preprocessing("validationA.csv", "valA.csv")#validationA.csv/valA.csv...validationF.csv/valF.csv
    train_start("trainA.csv")#trainA.csv...trainF.csv
    test_start("valA.csv",50)#valA.csv...valF.csv
    

第1组系数
[0.42587883 0.51497407 0.17546973 7.3249581 ]
0.39134892296706036
第2组系数
[ 0.57725822 -0.05757783  0.38185462  7.25766643]
0.30584968442681004
第3组系数
[0.13541851 0.71581159 0.08670534 6.83173107]
0.3305738246265509
第4组系数
[ 0.38083538 -2.88697789  2.9968059   8.30785995]
239.01071093129184
第5组系数
[ 1.20612349 -0.30203874  0.1748168   6.96477726]
0.36585125216057107
第6组系数
[-0.25        0.92013889 -1.95659722  4.04821181]
53.02557853425666
第7组系数
[-0.14267468 -0.85498377 -0.39260284  5.16548837]
0.563643656773942
第8组系数
[ 0.6982119  -0.8848334   0.27270842  6.83924259]
0.5744441067874914
第9组系数
[ 0.01249366  1.57638959 -0.230408    6.17101065]
0.4922549557370618
第10组系数
[ 0.20185575 -0.56681021 -1.74248095  4.31696446]
21.579275112699474
第11组系数
[-43.40540541  -9.41441441 126.06126126 113.12762162]
1.937494981948354e+85
第12组系数
[ 0.72907392 -1.48843188  1.42698602  7.77526303]
1.8988715061251507
第13组系数
[ 3.17299953 -0.82411391 -0.54663568  7.07467277]
0.8865237605035873
第14组系数
[ 3.5        -

5.222556996667387
第109组系数
[ 0.90841375 -0.43179896 -0.01926945  6.86912377]
0.3981853077625176
第110组系数
[-0.69101004 -0.04057228 -0.06525731  5.48093487]
0.5113842272280582
第111组系数
[ 1.53663004 -0.21794872  0.22502383  7.26155627]
0.3945257119472055
第112组系数
[-1.15528491  0.1722488  -0.44588811  4.74288653]
1.3654004726798041
第113组系数
[0.70093309 0.14900379 0.61413389 7.63112977]
0.3439659996985642
第114组系数
[0.23014865 0.19163235 1.00768999 7.79830994]
0.42191079866613035
第115组系数
[ 1.79204178e-05 -3.16362213e-01  6.48530235e-02  6.09368372e+00]
0.25321083599514804
第116组系数
[ 2.69722905e-01  2.88111177e-03 -5.55054656e-01  5.70639234e+00]
0.4815829573127443
第117组系数
[ 2.32518272 -0.73687708 -1.95338206  5.58342405]
30.334252141808623
第118组系数
[ 6.1875     1.125      2.2900641 11.83625  ]
0.9795649881779025
第119组系数
[-0.0762446  -0.16684098  0.28458374  6.69671001]
0.40150279098549224
第120组系数
[ 0.89852862 -0.58789401 -0.82047371  5.77271398]
0.971706979315137
第121组系数
[ 1.75918196 -0.89746425 -0.

512535.413689144
第215组系数
[ 0.94108713 -0.61659652 -0.51428093  5.98173894]
0.5024670496516591
第216组系数
[ 0.41815233  0.18300893 -1.41889046  5.39685651]
8.455748739088131
第217组系数
[ 0.64550588 -0.03754006  0.88425149  7.90242408]
0.4277029732070865
第218组系数
[-0.35817079 -0.89308742 -0.13695806  4.89344305]
0.49918295032924237
第219组系数
[0.60914761 0.0338755  0.64501406 7.2689289 ]
0.36707493151143167
第220组系数
[ 0.30846834 -2.2918696  -0.35200278  5.30140195]
11.388591632783976
第221组系数
[ 0.83323419 -0.39440809  0.11335845  7.17858153]
0.5163259474436706
第222组系数
[ 0.70434565 -0.97516971  0.8216114   7.23483391]
0.6745379954753579
第223组系数
[-0.02582573 -0.1913278   0.26096902  6.5055204 ]
0.28255790325411195
第224组系数
[ 0.46122793 -0.04465213  0.22354621  6.62610332]
0.30675708940771856
第225组系数
[0.09107782 0.01317906 0.1469058  6.77773402]
0.35403650131423187
第226组系数
[ 0.22844175 -0.25567322 -0.17821483  6.02701059]
0.26487171857178887
第227组系数
[-0.36175205 -0.90190793  0.07599587  5.39186044]
0.50

0.627287290489557
第322组系数
[ 0.32427088 -1.0502554   0.20504202  6.41446169]
0.6726578214570781
第323组系数
[ 0.8477746  -0.16807929  0.30986473  6.89608728]
0.3592211892919134
第324组系数
[0.83911355 0.44878697 0.75147818 8.30880824]
0.5359386789333659
第325组系数
[0.30912445 0.36249989 0.15918903 7.16150132]
0.37315724719233845
第326组系数
[1.22855216 0.46578752 0.23174904 7.76331452]
0.42823291526729157
第327组系数
[0.58437204 1.07686498 0.04574173 7.65216756]
0.6561163437411319
第328组系数
[ 1.26776389 -3.2222243  -1.02731211  4.38640616]
266.81718238728996
第329组系数
[ 1.72627373 -1.79220779  0.42        6.95352388]
2.6952235653495675
第330组系数
[ 0.49832748 -0.39305361  0.40473504  6.94900157]
0.32784452952152454
第331组系数
[ 0.19732882  0.07174837 -0.30754668  5.84720534]
0.33695786794197735
第332组系数
[ 0.37076556 -0.07373002  0.34350657  7.15209139]
0.32619481245239274
第333组系数
[ 0.24614671 -0.65291663  0.27534081  6.48236168]
0.37199697598789777
第334组系数
[ 4.65764706  0.16470588 -2.42996078  5.09804   ]
36.3427449

109614.25300645696
第429组系数
[0.23930724 0.65035139 0.21195535 7.48151536]
0.6014993227184923
第430组系数
[-1.18661828  6.22302867  4.43192579 15.05013507]
44.44596942346937
第431组系数
[ 1.22360449  0.44289029 -0.08817842  7.37204847]
0.44840174325769505
第432组系数
[ 1.2755243  -0.24940349  0.2449663   7.14380133]
0.3699109748608989
第433组系数
[-3.85395345 -8.77409202 11.66246264 15.39498551]
8706240503144560.0
第434组系数
[ 2.83987235  3.65045992  0.84862024 11.18490558]
1.9921079759944293
第435组系数
[0.60414053 0.4642409  1.8161857  8.84606023]
0.5906444300828494
第436组系数
[0.51445365 0.15359945 0.09119339 7.10049543]
0.33484777231732343
第437组系数
[ 0.02356071 -0.7201216   0.03097093  5.93958769]
0.37229605914294395
第438组系数
[ 0.41812164  0.38636364 -0.08200786  6.8914819 ]
0.3694495716175456
第439组系数
[ 1.94230306 -1.95452817  0.27192029  7.67386795]
9.704613625667449
第440组系数
[-4.20288729 -5.52355636 -0.61712743  1.17460085]
5956142.289910658
第441组系数
[-7.02083803 -5.07204249 -2.35024411 -5.4056929 ]
23416873422

...

7.520605371003248e+20
第7523组系数
[ 0.68683401 -0.34254192 -0.15483619  6.47719697]
0.313654831006423
第7524组系数
[ 0.2006706   0.07961513 -0.09972212  6.5859672 ]
0.36802773181344345
第7525组系数
[ 1.86171171 -0.42252252  0.09979279  7.15810532]
0.4467151747294725
第7526组系数
[-0.0640341   0.74329261 -0.12935291  6.32498699]
0.39011259441121565
第7527组系数
[ 0.07489289  0.07771411 -0.05910307  6.34838925]
0.2678035461484385
第7527组系数
[ 0.86703185  0.13266697 -0.05526494  6.97002507]
0.3114350752007312
第7528组系数
[-17.01878567   4.29134098   7.75132902  15.40466946]
2.716838660537127e+33
第7529组系数
[ 0.09524661 -0.06115713 -0.31024114  6.09994825]
0.42699000212132315
第7530组系数
[ 0.30058954 -0.00792501 -0.13936704  6.31403382]
0.25261248479898324
第7531组系数
[-0.32302     0.07402447  0.7347772   6.86888079]
0.4099798053157892
第7532组系数
[ 0.62011675 -2.45621913 -0.44607095  5.59413651]
26.57213354878455
第7533组系数
[-0.45833333  1.07896103 -0.10286536  5.97241454]
0.4954794998190809
第7534组系数
[-20.          -4.400361

0.3649682193569481
第7626组系数
[ 1.45192089  3.42782451  2.70516026 11.95439873]
1.5148662765563068
第7627组系数
[ 1.39285714e+00 -5.20943771e-03  1.80793030e-01  7.35278495e+00]
0.35820375325337295
第7628组系数
[-0.0220943   0.26740851 -0.11136358  6.32695051]
0.3411000086714026
第7629组系数
[ 0.43006341 -3.22491608  0.21857516  5.55046997]
471.0869352623597
第7630组系数
[-0.35331044 -0.05034232  0.10903439  5.81516771]
0.3352378469256737
第7631组系数
[ 1.69177681 -2.35649952 -0.09960314  6.58819085]
22.501709198912405
第7632组系数
[ -4.66666667   3.         -10.44927536 -10.61710145]
1.3741042547695498e+21
第7633组系数
[ 0.71276457 -0.0094562   0.1603781   7.06492806]
0.2815545258987705
第7634组系数
[ 1.43811468 -0.03281963 -0.07771338  7.09883606]
0.3689660136998687
第7635组系数
[ 0.67512564  0.23620962 -0.04976441  6.91374489]
0.3003108850420994
第7636组系数
[17.29383386 16.45168099 38.22870496 74.00999095]
1.0812915335004006e+19
第7637组系数
[-0.06001606 -0.11039743  0.65064231  6.74087214]
0.34749052218279486
第7638组系数
[ 0.723

51.06020632893585
第7730组系数
[ 0.74912328 -1.30294038 -1.11109717  4.40905565]
1.393012684945271
第7731组系数
[ 1.13150392  3.52610206  7.26837963 21.10510888]
923.0009663532142
第7732组系数
[ 0.02530254 -0.47058824  0.07707595  5.9173587 ]
0.3011803661498455
第7733组系数
[ 4.89311325 -3.95636134 -3.11716953  3.22300095]
7031.576220958477
第7734组系数
[ 4.80888819 -7.89606515 -2.88146088  1.11452059]
139479962114.78848
第7735组系数
[1.80780832 0.31692603 0.48125043 8.12512922]
0.462386351403777
第7736组系数
[-0.17344654  1.01700874  0.07260169  6.06189891]
0.47522857264666296
第7737组系数
[-0.61034611 -1.62965389 -1.34381762  3.39073227]
8.041693253467413
第7738组系数
[ 3.89103927  7.49604932 -0.35470277 10.87553173]
2.64966496242921
第7739组系数
[ 0.04478185  0.16003372 -0.19402668  5.85775295]
0.32835622257274677
第7740组系数
[-0.25622789 -0.33783522 -0.09857604  5.84281389]
0.41301806534284147
第7741组系数
[-46.875041    22.47195802  54.35060676  47.32739915]
1.5312663048756277e+90
第7742组系数
[0.62508445 0.46903393 0.29196482 7.5

0.36061320499641913
第7834组系数
[0.1622403  1.50993142 0.21168439 7.2763691 ]
0.44787926037647624
第7835组系数
[ 0.83982202 -0.67754449  0.71879588  7.28267583]
0.4923531160307612
第7836组系数
[-10.4258085  -13.52346227   2.7878884    4.79466709]
2.566248448647116e+25
第7837组系数
[-5.14185672 -0.4923136   5.7973278   4.88757245]
37220.80213644602
第7838组系数
[-0.249016   -0.0406556  -0.03275243  5.74980871]
0.3136419844095794
第7839组系数
[-0.68761246 -1.38096598  3.20410728  9.47580932]
7.030818161174977
第7840组系数
[1.35260282 0.0808168  1.40030678 8.86013632]
0.5977397103864742
第7841组系数
[-0.08017241 -0.76637931 -0.20279877  5.3950444 ]
0.4105317421519312
第7842组系数
[ -97.58423913 -228.88315217  274.04619565  103.71478261]
第7842组系数
[ 0.14717327 -0.26332173  0.3018843   6.86257568]
0.37808915769993423
第7843组系数
[-0.21865079 -0.44943097 -0.01911171  5.65013632]
0.32532525325703976
第7844组系数
[ 0.4635122  -1.74144715  0.68286557  6.92132785]
3.3436040235251463
第7845组系数
[-0.60989011 -3.07142857  5.71428571 11.401593

1.1244057907982032
第7939组系数
[0.88384731 0.99736621 0.00906989 7.74133322]
0.644451523908744
第7940组系数
[ 0.56570353 -0.05939166  0.00868163  6.90521435]
0.3229254377608969
第7941组系数
[ 0.18632206 -0.80956694 -0.04223743  5.94651812]
0.4015490619479883
第7942组系数
[0.59707857 0.9099305  0.13316386 7.49004278]
0.4371431744520331
第7943组系数
[ 0.50669547 -0.26241529  0.25180194  7.06816964]
0.3568819545833243
第7944组系数
[1.19479284 0.41469077 0.3855419  8.03486796]
0.50554851843378
第7945组系数
[ 0.01165657 -0.12713058 -0.18094098  5.92597443]
0.28095966505488806
第7946组系数
[ 1.45559846 -3.2020592  -2.36164736  2.35480051]
198.55399082297973
第7947组系数
[ 0.72349654 -0.28465438 -0.13952291  6.45838734]
0.2926182667581556
第7948组系数
[ 0.10786058 -0.04388216  0.37585349  6.77345749]
0.2644207798345315
第7949组系数
[ 0.94071396 -0.18857941 -0.21998414  6.63369318]
0.34397839114012857
第7950组系数
[-0.03044676 -0.36525948  0.36295144  6.16932284]
0.35653668222893203
第7951组系数
[2.26315136 0.26457312 0.30339154 8.03219496]
0.